### Basic SQL

In [2]:
!pip install --quiet ipython-sql

fastlmm 0.2.32 requires dill, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
grpcio 1.11.0 has requirement protobuf>=3.5.0.post1, but you'll have protobuf 3.4.1 which is incompatible.


In [3]:
%load_ext sql
import pandas as pd
%sql sqlite://

'Connected: @None'

In [4]:
# For compatibility across multiple platforms
import os
#IB = os.environ.get('INSTABASE_URI',None) is not None
#open = ib.open if IB else open

In [9]:
!head -5 Cities.csv

city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.10
Abisko,Sweden,63.35,18.83,0.20
Adana,Turkey,36.99,35.32,18.67


In [17]:
# Load tables from CSV files
# Cities
with open('Cities.csv','rU') as f:
    Cities = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Cities;
%sql persist Cities
# Countries
with open('Countries.csv','rU') as f:
    Countries = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Countries;
%sql persist Countries
# Players
with open('Players.csv','rU') as f:
    Players = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Players;
%sql persist Players
# Teams
with open('Teams.csv','rU') as f:
    Teams = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql drop table if exists Teams;
%sql persist Teams

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.
 * sqlite://


u'Persisted teams'

#### Look at sample of Cities and Countries tables

In [18]:
%%sql
select * from Cities limit 5

 * sqlite://
Done.


[(u'Aalborg', u'Denmark', 57.03, 9.92, 7.52),
 (u'Aberdeen', u'United Kingdom', 57.17, -2.08, 8.1),
 (u'Abisko', u'Sweden', 63.35, 18.83, 0.2),
 (u'Adana', u'Turkey', 36.99, 35.32, 18.67),
 (u'Albacete', u'Spain', 39.0, -1.87, 12.62)]

In [19]:
%%sql
select * from Countries limit 5

 * sqlite://
Done.


[(u'Albania', 2.9, u'no', u'yes'),
 (u'Andorra', 0.07, u'no', u'no'),
 (u'Austria', 8.57, u'yes', u'no'),
 (u'Belarus', 9.48, u'no', u'no'),
 (u'Belgium', 11.37, u'yes', u'yes')]

### Basic Select statement
Select columns  
From tables  
Where condition  

*Find all countries not in the EU*

In [22]:
%%sql
select country
from Countries
where EU = 'no'

 * sqlite://
Done.


[(u'Albania',),
 (u'Andorra',),
 (u'Belarus',),
 (u'Bosnia and Herzegovina',),
 (u'Iceland',),
 (u'Kosovo',),
 (u'Liechtenstein',),
 (u'Macedonia',),
 (u'Moldova',),
 (u'Montenegro',),
 (u'Norway',),
 (u'Serbia',),
 (u'Switzerland',),
 (u'Turkey',),
 (u'Ukraine',)]

*Find all cities with temperature between -5 and 5; return city, country, and temperature*

In [23]:
%%sql
select city, country, temperature
from Cities
where temperature > -5 and temperature < 5

 * sqlite://
Done.


[(u'Abisko', u'Sweden', 0.2),
 (u'Augsburg', u'Germany', 4.54),
 (u'Bergen', u'Norway', 1.75),
 (u'Bodo', u'Norway', 4.5),
 (u'Helsinki', u'Finland', 4.19),
 (u'Innsbruck', u'Austria', 4.54),
 (u'Kiruna', u'Sweden', -2.2),
 (u'Orsha', u'Belarus', 4.93),
 (u'Oslo', u'Norway', 2.32),
 (u'Oulu', u'Finland', 1.45),
 (u'Salzburg', u'Austria', 4.62),
 (u'Tallinn', u'Estonia', 4.82),
 (u'Tampere', u'Finland', 3.59),
 (u'Tartu', u'Estonia', 4.36),
 (u'Trondheim', u'Norway', 4.53),
 (u'Turku', u'Finland', 4.72),
 (u'Uppsala', u'Sweden', 4.17)]

### Ordering

*Modify previous query to sort by temperature*

*Modify previous query to sort by country, then temperature descending*

### <font color = 'green'>Your Turn</font>

*Find all countries with no coastline and with population > 9. Return the country and population, in descending order of population.*

In [28]:
%%sql
select * from Countries 
where coastline = 'no'
and population > 9
order by population desc

 * sqlite://
Done.


[(u'Czech Republic', 10.55, u'yes', u'no'),
 (u'Hungary', 9.82, u'yes', u'no'),
 (u'Belarus', 9.48, u'no', u'no')]

### Multiple tables in From clause - Joins

*Find all cities with longitude < 10 not in the EU, return city and longitude*

In [29]:
%%sql
select city, longitude
from Cities, Countries
where Cities.country = Countries.country
and longitude < 10 and EU = 'no'

 * sqlite://
Done.


[(u'Andorra', 1.52),
 (u'Basel', 7.59),
 (u'Bergen', 5.32),
 (u'Geneva', 6.14),
 (u'Stavanger', 5.68),
 (u'Zurich', 8.56)]

*Modify previous query to also return country (error then fix)*

*Find all cities with latitude < 50 in a country with population < 5; return city, country, and population, sorted by country*

In [30]:
%%sql
select city, Cities.country, population
from Cities, Countries
where Cities.country = Countries.country
and latitude < 50 and population < 5
order by Cities.country

 * sqlite://
Done.


[(u'Elbasan', u'Albania', 2.9),
 (u'Andorra', u'Andorra', 0.07),
 (u'Sarajevo', u'Bosnia and Herzegovina', 3.8),
 (u'Rijeka', u'Croatia', 4.23),
 (u'Split', u'Croatia', 4.23),
 (u'Skopje', u'Macedonia', 2.08),
 (u'Balti', u'Moldova', 4.06),
 (u'Chisinau', u'Moldova', 4.06),
 (u'Podgorica', u'Montenegro', 0.63),
 (u'Ljubljana', u'Slovenia', 2.07)]

#### Inner Join -- just FYI

*Same query as above*

In [31]:
%%sql
select city, Cities.country, population
from Cities inner join Countries
     on Cities.country = Countries.country
where latitude < 50 and population < 5
order by Cities.country

 * sqlite://
Done.


[(u'Elbasan', u'Albania', 2.9),
 (u'Andorra', u'Andorra', 0.07),
 (u'Sarajevo', u'Bosnia and Herzegovina', 3.8),
 (u'Rijeka', u'Croatia', 4.23),
 (u'Split', u'Croatia', 4.23),
 (u'Skopje', u'Macedonia', 2.08),
 (u'Balti', u'Moldova', 4.06),
 (u'Chisinau', u'Moldova', 4.06),
 (u'Podgorica', u'Montenegro', 0.63),
 (u'Ljubljana', u'Slovenia', 2.07)]

### Select *

*Modify previous queries to return all attributes*

### <font color = 'green'>Your Turn</font>

*Find all cities with latitude > 45 in a country with no coastline and with population > 9. Return the city, country, latitude, and whether it's in the EU.*

In [35]:
%%sql
select city, Cities.country, latitude, EU
from Cities, Countries
where Cities.country = Countries.country
and latitude > 45 and population > 9 and coastline = 'no'
order by Cities.country

 * sqlite://
Done.


[(u'Brest', u'Belarus', 52.1, u'no'),
 (u'Hrodna', u'Belarus', 53.68, u'no'),
 (u'Mazyr', u'Belarus', 52.05, u'no'),
 (u'Minsk', u'Belarus', 53.9, u'no'),
 (u'Orsha', u'Belarus', 54.52, u'no'),
 (u'Pinsk', u'Belarus', 52.13, u'no'),
 (u'Brno', u'Czech Republic', 49.2, u'yes'),
 (u'Ostrava', u'Czech Republic', 49.83, u'yes'),
 (u'Prague', u'Czech Republic', 50.08, u'yes'),
 (u'Budapest', u'Hungary', 47.5, u'yes'),
 (u'Debrecen', u'Hungary', 47.53, u'yes'),
 (u'Gyor', u'Hungary', 47.7, u'yes'),
 (u'Szeged', u'Hungary', 46.25, u'yes')]

### Aggregation and Grouping

*Find the average temperature for all cities*

In [36]:
%%sql
select avg(temperature)
from Cities

 * sqlite://
Done.


[(9.497840375586858,)]

*Modify previous query to find average temperature of cities with latitude > 55*

In [38]:
%%sql
select avg(temperature)
from Cities
where latitude > 55

 * sqlite://
Done.


[(4.985185185185185,)]

*Modify previous query to also find minimum and maxiumum temperature of cities with latitude > 55*

In [39]:
%%sql
select min(temperature), max(temperature)
from Cities
where latitude > 55

 * sqlite://
Done.


[(-2.2, 8.6)]

*Modify previous query to return number of cities with latitude > 55*

In [41]:
%%sql
select count(*)
from Cities
where latitude > 55

 * sqlite://
Done.


[(27,)]

*Rename result column as northerns*

In [42]:
%%sql
select count(*) as northerns
from Cities
where latitude > 55

 * sqlite://
Done.


[(27,)]

*Find the minimum and maximum temperature of cities in the EU (then not in the EU)*

In [47]:
%%sql
select min(temperature), max(temperature)
from Cities, Countries
where Cities.country = Countries.country
and EU = 'yes'

 * sqlite://
Done.


[(-2.2, 17.9)]

In [48]:
%%sql
select min(temperature), max(temperature)
from Cities, Countries
where Cities.country = Countries.country
and EU = 'no'

 * sqlite://
Done.


[(1.75, 18.67)]

### <font color = 'green'>Your Turn</font>

*Find the number of cities with latitude > 45 in countries with no coastline and with population > 9; also return the minimum and maximum latitude among those cities*

In [51]:
%%sql
--select min(latitude), max(latitude)
select count(*)
from Cities, Countries
where Cities.country = Countries.country
and latitude > 45 and coastline = 'no' and population > 9

 * sqlite://
Done.


[(13,)]

In [52]:
%%sql
select min(latitude), max(latitude)
from Cities, Countries
where Cities.country = Countries.country
and latitude > 45 and coastline = 'no' and population > 9

 * sqlite://
Done.


[(46.25, 54.52)]

*Find the average temperature for each country*

In [53]:
%%sql
select country, avg(temperature)
from Cities
group by country

 * sqlite://
Done.


[(u'Albania', 15.18),
 (u'Andorra', 9.6),
 (u'Austria', 6.144),
 (u'Belarus', 5.946666666666666),
 (u'Belgium', 9.65),
 (u'Bosnia and Herzegovina', 9.6),
 (u'Bulgaria', 10.44),
 (u'Croatia', 10.865),
 (u'Czech Republic', 7.8566666666666665),
 (u'Denmark', 7.625),
 (u'Estonia', 4.59),
 (u'Finland', 3.4875),
 (u'France', 10.151111111111112),
 (u'Germany', 7.869285714285714),
 (u'Greece', 16.9025),
 (u'Hungary', 9.6025),
 (u'Ireland', 9.299999999999999),
 (u'Italy', 13.474666666666668),
 (u'Latvia', 5.27),
 (u'Lithuania', 6.1433333333333335),
 (u'Macedonia', 9.36),
 (u'Moldova', 8.415),
 (u'Montenegro', 9.99),
 (u'Netherlands', 8.756666666666668),
 (u'Norway', 3.7260000000000004),
 (u'Poland', 7.250000000000002),
 (u'Portugal', 14.469999999999999),
 (u'Romania', 9.224444444444444),
 (u'Serbia', 9.85),
 (u'Slovakia', 8.48),
 (u'Slovenia', 9.27),
 (u'Spain', 14.238333333333332),
 (u'Sweden', 3.5866666666666673),
 (u'Switzerland', 7.253333333333333),
 (u'Turkey', 11.726666666666665),
 (u'Ukraine', 7.420000000000002),
 (u'United Kingdom', 8.649999999999999)]

*Modify previous query to sort by descending average temperature*

In [55]:
%%sql
select country, avg(temperature)
from Cities
group by country
order by avg(temperature) desc

 * sqlite://
Done.


[(u'Greece', 16.9025),
 (u'Albania', 15.18),
 (u'Portugal', 14.469999999999999),
 (u'Spain', 14.238333333333332),
 (u'Italy', 13.474666666666668),
 (u'Turkey', 11.726666666666665),
 (u'Croatia', 10.865),
 (u'Bulgaria', 10.44),
 (u'France', 10.151111111111112),
 (u'Montenegro', 9.99),
 (u'Serbia', 9.85),
 (u'Belgium', 9.65),
 (u'Hungary', 9.6025),
 (u'Andorra', 9.6),
 (u'Bosnia and Herzegovina', 9.6),
 (u'Macedonia', 9.36),
 (u'Ireland', 9.299999999999999),
 (u'Slovenia', 9.27),
 (u'Romania', 9.224444444444444),
 (u'Netherlands', 8.756666666666668),
 (u'United Kingdom', 8.649999999999999),
 (u'Slovakia', 8.48),
 (u'Moldova', 8.415),
 (u'Germany', 7.869285714285714),
 (u'Czech Republic', 7.8566666666666665),
 (u'Denmark', 7.625),
 (u'Ukraine', 7.420000000000002),
 (u'Switzerland', 7.253333333333333),
 (u'Poland', 7.250000000000002),
 (u'Austria', 6.144),
 (u'Lithuania', 6.1433333333333335),
 (u'Belarus', 5.946666666666666),
 (u'Latvia', 5.27),
 (u'Estonia', 4.59),
 (u'Norway', 3.7260000000000004),
 (u'Sweden', 3.5866666666666673),
 (u'Finland', 3.4875)]

*Modify previous query to show countries only*

In [56]:
%%sql
select country
from Cities
group by country
order by avg(temperature) desc

 * sqlite://
Done.


[(u'Greece',),
 (u'Albania',),
 (u'Portugal',),
 (u'Spain',),
 (u'Italy',),
 (u'Turkey',),
 (u'Croatia',),
 (u'Bulgaria',),
 (u'France',),
 (u'Montenegro',),
 (u'Serbia',),
 (u'Belgium',),
 (u'Hungary',),
 (u'Andorra',),
 (u'Bosnia and Herzegovina',),
 (u'Macedonia',),
 (u'Ireland',),
 (u'Slovenia',),
 (u'Romania',),
 (u'Netherlands',),
 (u'United Kingdom',),
 (u'Slovakia',),
 (u'Moldova',),
 (u'Germany',),
 (u'Czech Republic',),
 (u'Denmark',),
 (u'Ukraine',),
 (u'Switzerland',),
 (u'Poland',),
 (u'Austria',),
 (u'Lithuania',),
 (u'Belarus',),
 (u'Latvia',),
 (u'Estonia',),
 (u'Norway',),
 (u'Sweden',),
 (u'Finland',)]

*Find the average temperature for cities in countries with and without coastline*

In [57]:
%%sql
select coastline, avg(temperature)
from Cities, Countries
where Cities.country = Countries.country
group by coastline

 * sqlite://
Done.


[(u'no', 7.748000000000001), (u'yes', 9.784699453551914)]

*Modify previous query to find the average temperature for cities in the EU and not in the EU, then all combinations of coastline and EU*

In [60]:
%%sql
select EU, avg(temperature)
from Cities, Countries
where Cities.country = Countries.country
group by EU

 * sqlite://
Done.


[(u'no', 9.030476190476188), (u'yes', 9.694133333333333)]

In [61]:
%%sql
select coastline, EU, avg(temperature)
from Cities, Countries
where Cities.country = Countries.country
group by coastline, EU

 * sqlite://
Done.


[(u'no', u'no', 7.67375),
 (u'no', u'yes', 7.832857142857144),
 (u'yes', u'no', 9.492340425531914),
 (u'yes', u'yes', 9.885735294117652)]

*Modify previous query to only include cities with latitude < 50, then latitude < 40*

In [62]:
%%sql
select coastline, EU, avg(temperature)
from Cities, Countries
where Cities.country = Countries.country
and latitude < 50
group by coastline, EU

 * sqlite://
Done.


[(u'no', u'no', 8.709999999999999),
 (u'no', u'yes', 7.816153846153847),
 (u'yes', u'no', 10.675675675675675),
 (u'yes', u'yes', 11.928076923076924)]

In [63]:
%%sql
select coastline, EU, avg(temperature)
from Cities, Countries
where Cities.country = Countries.country
and latitude < 40
group by coastline, EU

 * sqlite://
Done.


[(u'yes', u'no', 11.766875), (u'yes', u'yes', 16.320555555555554)]

### <font color = 'green'>Your Turn</font>

*For each country in the EU, find the latitude of the northernmost city in the country, i.e., the maximum latitude. Return the country and its maximum latitude, in descending order of maximum latitude.*

In [ ]:
%%sql
YOUR QUERY HERE

### The Limit clause

*Return any three countries with population > 20*

In [ ]:
%%sql
select country
from Countries
where population > 20
limit 3

*Find the ten coldest cities*

In [ ]:
%%sql
select city, temperature
from Cities
order by temperature
limit 10

### <font color = 'green'>Your Turn - Basic SQL on World Cup Data</font>

#### Look at sample of Players and Teams tables

In [ ]:
%%sql
select * from Players limit 5

In [ ]:
%%sql
select * from Teams limit 5

*1)  What player on a team with “ia” in the team name played less than 200 minutes and made more than 100 passes? Return the player surname. Note: To check if attribute A contains string S use "A like '%S%'"*

In [ ]:
%%sql
YOUR QUERY HERE

*2) Find all players who made more than 20 shots. Return all player information in descending order of shots made.*

In [ ]:
%%sql
YOUR QUERY HERE

*3) Find the goalkeepers of teams that played more than four games. List the surname of the goalkeeper, the team, and the number of minutes the goalkeeper played.*

In [ ]:
%%sql
YOUR QUERY HERE

*4) How many players who play on a team with ranking <10 played more than 350 minutes? Return one number in a column named 'superstar'.*

In [ ]:
%%sql
YOUR QUERY HERE

*5) What is the average number of passes made by forwards? By midfielders? Write one query that gives both values.*

In [ ]:
%%sql
YOUR QUERY HERE

*6) Which team has the highest average number of passes per minute played? Return the team and average passes per minute. Hints: (1) You can compute a team's average number of passes per minute played by dividing the total number of passes by the total number of minutes. To force floating point division, multiply one operand by 1.0. (2) Consider using Limit.*

In [ ]:
%%sql
YOUR QUERY HERE